In [1]:
model_path = "hf-internal-testing/Mixtral-tiny"
token_path = "mistralai/Mixtral-8x7B-v0.1"

import numpy as np
from transformers import AutoTokenizer, MixtralModel, MixtralConfig, FlaxMixtralModel
config = MixtralConfig.from_pretrained(model_path)
config.output_router_logits = True
config.output_attentions = True
config.output_hidden_states = True
# config.max_position_embeddings = 4096

/home/adrien/Documents/flax/FlaxMixtral/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = MixtralModel.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(token_path)

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)

In [ ]:
x.hidden_states

In [2]:
model = FlaxMixtralModel.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(token_path)

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

y = model(**inputs)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


: 

In [ ]:
y.hidden_states

In [ ]:
to_keep = []
for i in range(len(x.attentions)):
        for j in range(len(x.attentions[i])):
                for k in range(len(x.attentions[i][j])):
                        for l in range(len(x.attentions[i][j][k])):
                                to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j][k][l].detach().cpu()) - np.array(y.attentions[i][j][k][l]))))

# for i in range(len(x.last_hidden_state)):
#         for j in range(len(x.last_hidden_state[i])):
#                                 to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j].detach().cpu()) - np.array(y.attentions[i][j]))))

to_keep.append(np.amax(np.abs(np.array(x.attentions[0].detach().cpu()) - np.array(y.attentions[0]))))


for i in range(len(x.hidden_states)):
        for j in range(len(x.hidden_states[i])):
                for k in range(len(x.hidden_states[i][j])):
                                to_keep.append(np.amax(np.abs(np.array(x.hidden_states[i][j][k].detach().cpu()) - np.array(y.hidden_states[i][j][k]))))


In [ ]:
to_keep = []
for i in range(len(x.attentions)):
        for j in range(len(x.attentions[i])):
                for k in range(len(x.attentions[i][j])):
                        for l in range(len(x.attentions[i][j][k])):
                                to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j][k][l].detach().cpu()) - np.around(np.array(y.attentions[i][j][k][l]), 4))))

# for i in range(len(x.last_hidden_state)):
#         for j in range(len(x.last_hidden_state[i])):
#                                 to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j].detach().cpu()) - np.array(y.attentions[i][j]))))

to_keep.append(np.amax(np.abs(np.array(x.attentions[0].detach().cpu()) - np.around(np.array(y.attentions[0]), 4))))


for i in range(len(x.hidden_states)):
        for j in range(len(x.hidden_states[i])):
                for k in range(len(x.hidden_states[i][j])):
                                to_keep.append(np.amax(np.abs(np.array(x.hidden_states[i][j][k].detach().cpu()) - np.around(np.array(y.hidden_states[i][j][k]), 4))))
